In [ ]:
cd /home/drew/NBOH

In [ ]:
import asyncio
import nest_asyncio
import os

nest_asyncio.apply() #https://github.com/jupyter/notebook/issues/3397#issuecomment-419386811

parent_dir = os.getcwd()
nsimulations = 7
first_sim_n = 2
nproc = 64
proc_per_sim = nproc//nsimulations
stride = 2

def grompp(directory, mdp, i, o,  is_continuation, maxwarn=1):
    parameters = ['gmx', 'grompp', 
                  '-f', f'{mdp}.mdp', 
                  '-c', f'{i}.pdb', 
                  '-r', 'fullSystemStart.pdb', 
                  '-p', 'topol.top', 
                  '-o', f'{o}.tpr', 
                  '-n',
                  '-maxwarn', f'{maxwarn}']
    if is_continuation:
        parameters.extend(['-t', f'{i}.trr'])
    param_str = " ".join(parameters)
    return f'cd {parent_dir}; cd {directory}; {param_str}'
    #return await asyncio.create_subprocess_shell(cmd, stdout=asyncio.subprocess.DEVNULL, stderr=asyncio.subprocess.DEVNULL)
    #return subp.run(parameters, capture_output=True)

def mdrun(directory, i, nt, gpu_id, pinoffset):
    parameters = ['gmx', 'mdrun', 
                     '-nt', f'{nt}', 
                     '-gpu_id', f'{gpu_id}', 
                     '-ntmpi', '1', 
                     '-pin', 'on', 
                     '-pinstride', f'{stride}', 
                     '-pinoffset', f'{pinoffset}', 
                     '-deffnm', f'{i}', 
                     '-c', f'{i}.pdb', 
                     '-nice', '+10']
    return f'{" ".join(parameters)}'
    #return await asyncio.create_subprocess_shell(cmd, stdout=asyncio.subprocess.DEVNULL, stderr=asyncio.subprocess.DEVNULL)
    #return subp.run(               capture_output=True)

def prep_and_run_phase(directory, mdp, i, o, nt, gpu_id, pinoffset, maxwarn=1, is_continuation=False):
    return f'{grompp(directory, mdp, i, o, is_continuation, maxwarn=maxwarn)}; {mdrun(directory, o, nt, gpu_id, pinoffset)}'
    
def run_sim(sim_dex=0, first_directory=0):
    offset = stride * sim_dex * proc_per_sim
    gpu_id = 0 if sim_dex < nsimulations / 2 else 1 #we only have 2 gpus 0 and 1 otherwise a binary check wouldn't suffice
    directory = f'run{sim_dex + first_directory:02}'
    return ';'.join([
        #prep_and_run_phase(directory, 'minim_restrained', 'fullSystemStart', 'em', proc_per_sim, gpu_id, offset),
        #prep_and_run_phase(directory, 'minim', 'em', 'em2', proc_per_sim, gpu_id, offset),
        prep_and_run_phase(directory, 'nvt', 'em2', 'nvt', proc_per_sim, gpu_id, offset),
        prep_and_run_phase(directory, 'npt', 'nvt', 'npt', proc_per_sim, gpu_id, offset, maxwarn=2, is_continuation=True),
        prep_and_run_phase(directory, 'md', 'npt', 'md_0_1', proc_per_sim, gpu_id, offset, is_continuation=True)])

async def run(cmd, do_stdout=False, do_stderr=False):
    proc = await asyncio.create_subprocess_shell(cmd, stdout=asyncio.subprocess.PIPE, stderr=asyncio.subprocess.PIPE)
    #await proc.wait()
    stdout, stderr = await proc.communicate()
    print(f'[{cmd!r} exited with {proc.returncode}]')
    if do_stdout and stdout:
        print(f'[stdout]\n{stdout.decode()}')
    if do_stderr and stderr:
        print(f'[stderr]\n{stderr.decode()}')

async def main():
    await asyncio.gather(*[run(run_sim(n, first_sim_n), do_stderr=True) for n in range(nsimulations)])
    
asyncio.run(main())
